In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
import random
from collections import Counter

from generator import generateCliffordCircuit
# from execution.transpiler import transpileListToQiskitCircuit

In [2]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [3]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['I', 'H', 'X', 'I', 'X'], ['CNOT_T', 'I', 'CNOT_C', 'I', 'I'], ['S', 'I', 'H', 'H', 'S'], ['CNOT_T', 'I', 'CNOT_C', 'I', 'I'], ['Z', 'X', 'I', 'H', 'Z']]
     ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌───┐
q_0: ┤ I ├─░─┤ X ├─░─┤ S ├─░─┤ X ├─░─┤ Z ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░ └─┬─┘ ░ ├───┤
q_1: ┤ H ├─░───┼───░─┤ I ├─░───┼───░─┤ X ├
     ├───┤ ░   │   ░ ├───┤ ░   │   ░ ├───┤
q_2: ┤ X ├─░───■───░─┤ H ├─░───■───░─┤ I ├
     ├───┤ ░       ░ ├───┤ ░       ░ ├───┤
q_3: ┤ I ├─░───────░─┤ H ├─░───────░─┤ H ├
     ├───┤ ░       ░ ├───┤ ░       ░ ├───┤
q_4: ┤ X ├─░───────░─┤ S ├─░───────░─┤ Z ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [4]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")


In [5]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('YXZZI')

In [6]:
def split_circuit_by_barrier(circuit):
    qasm = circuit.qasm()
    prelude = []
    circuits = [[]]
    for line in qasm.splitlines():
        if any([line.startswith(t) for t in ['OPENQASM', 'include', 'qreg', 'creg']]):
            prelude.append(line)
        elif line.startswith('barrier'):
            circuits.append([])
        else:
            circuits[-1].append(line)
    circuits_with_prelude = [prelude+circuit for circuit in circuits]
    for circuit_with_prelude in circuits_with_prelude:
        yield qk.QuantumCircuit.from_qasm_str('\n'.join(circuit_with_prelude))

# for subcircuit in (split_circuit_by_barrier(qiskitCircuit)):
#     print(subcircuit)

In [9]:
def G_twirling(circuit):
    '''This function performs G-twirling on a circuit. It randomly chooses a Pauli from {X,Y,Z,I} and prepends it before a layer, then appends P'=CPC^T after the layer.}
    Inputs: circuit - A Qiskit Circuit object.
    Outputs: new_circuit - A Qiskit Circuit object.'''
    num_qubits = circuit.num_qubits
    new_circuit = qk.QuantumCircuit(num_qubits)
    for index, subcircuit in enumerate(split_circuit_by_barrier(circuit)):
        # if len(subcircuit.data) == 1:
        #     operated_qubits = [subcircuit.find_bit(q).index for q in subcircuit.data[0].qubits]
        # else:
        #     operated_qubits = list(range(num_qubits))
        pauli_str = ''.join(random.choice(["X","Y","Z","I"]) for _ in range(num_qubits))
        pauli = qk.quantum_info.Pauli(pauli_str)
        new_circuit = new_circuit.compose(pauli,qubits=range(num_qubits))
        new_circuit = new_circuit.compose(subcircuit,qubits=range(num_qubits))
        new_circuit = new_circuit.compose(Clifford_Permute(subcircuit,pauli).to_instruction(),qubits = range(num_qubits))
        if index != len(list(split_circuit_by_barrier(circuit)))-1:
            new_circuit.barrier()
    return new_circuit


In [10]:
G_twirling(qiskitCircuit).draw()

┌───────────────┐┌───┐┌───────────────┐ ░ ┌───────────────┐┌───┐»
q_0: ┤0              ├┤ I ├┤0              ├─░─┤0              ├┤ X ├»
     │               │├───┤│               │ ░ │               │└─┬─┘»
q_1: ┤1              ├┤ H ├┤1              ├─░─┤1              ├──┼──»
     │               │├───┤│               │ ░ │               │  │  »
q_2: ┤2 Pauli(YIYIZ) ├┤ X ├┤2 Pauli(YIYIZ) ├─░─┤2 Pauli(XIZII) ├──■──»
     │               │├───┤│               │ ░ │               │     »
q_3: ┤3              ├┤ I ├┤3              ├─░─┤3              ├─────»
     │               │├───┤│               │ ░ │               │     »
q_4: ┤4              ├┤ X ├┤4              ├─░─┤4              ├─────»
     └───────────────┘└───┘└───────────────┘ ░ └───────────────┘     »
«     ┌───────────────┐ ░ ┌───────────────┐┌───┐┌───────────────┐ ░ »
«q_0: ┤0              ├─░─┤0              ├┤ S ├┤0              ├─░─»
«     │               │ ░ │               │├───┤│               │ ░ »
«q_1: ┤1              ├─░─┤1              ├┤ I ├┤1              ├─░─»
«     │               │ ░ │               │├───┤│               │ ░ »
«q_2: ┤2 Pauli(XIZII) ├─░─┤2 Pauli(IIXZX) ├┤ H ├┤2 Pauli(IIZZY) ├─░─»
«     │               │ ░ │               │├───┤│               │ ░ »
«q_3: ┤3              ├─░─┤3              ├┤ H ├┤3              ├─░─»
«     │               │ ░ │               │├───┤│               │ ░ »
«q_4: ┤4              ├─░─┤4              ├┤ S ├┤4              ├─░─»
«     └───────────────┘ ░ └───────────────┘└───┘└───────────────┘ ░ »
«     ┌───────────────┐┌───┐┌─────────┐ ░ ┌───────────────┐┌───┐»
«q_0: ┤0              ├┤ X ├┤0        ├─░─┤0              ├┤ Z ├»
«     │               │└─┬─┘│         │ ░ │               │├───┤»
«q_1: ┤1              ├──┼──┤1        ├─░─┤1              ├┤ X ├»
«     │               │  │  │         │ ░ │               │├───┤»
«q_2: ┤2 Pauli(ZXYXY) ├──■──┤2 -ZXXXZ ├─░─┤2 Pauli(XXIIY) ├┤ I ├»
«     │               │     │         │ ░ │               │├───┤»
«q_3: ┤3              ├─────┤3        ├─░─┤3              ├┤ H ├»
«     │               │     │         │ ░ │               │├───┤»
«q_4: ┤4              ├─────┤4        ├─░─┤4              ├┤ Z ├»
«     └───────────────┘     └─────────┘ ░ └───────────────┘└───┘»
«     ┌───────────────┐
«q_0: ┤0              ├
«     │               │
«q_1: ┤1              ├
«     │               │
«q_2: ┤2 Pauli(XZIIY) ├
«     │               │
«q_3: ┤3              ├
«     │               │
«q_4: ┤4              ├
«     └───────────────┘

In [ ]:
circuit_ensemble = [G_twirling(qiskitCircuit) for _ in range(10)]
# for circ in circuit_ensemble:
#     print(circ)

In [ ]:
def random_choices_with_counts(items, n):
    """
    Randomly choose with replacement n items from a list and return a dictionary
    with the counts of each chosen item.
    
    Parameters:
        items (list): List of items to choose from.
        n (int): Number of items to choose.
    
    Returns:
        dict: Dictionary with the counts of each chosen item.
    """
    chosen_items = random.choices(items, k=n)
    counts = Counter(chosen_items)
    return dict(counts)

In [ ]:
random_choices_with_counts([circ.qasm() for circ in circuit_ensemble],1000)

{'OPENQASM 2.0;\ninclude "qelib1.inc";\ngate pauli(param0) q0,q1,q2,q3,q4 { y q0; x q1; x q3; z q4; }\ngate gate__ZXIXY q0,q1,q2,q3,q4 { pauli(ZXIXY) q0,q1,q2,q3,q4; }\ngate pauli_140709000663440(param0) q0,q1,q2,q3,q4 { x q0; x q1; z q2; z q3; x q4; }\ngate pauli_140709000669776(param0) q0,q1,q2,q3,q4 { x q0; x q1; z q2; z q3; x q4; }\ngate pauli_140709000674448(param0) q0,q1,q2,q3,q4 { z q0; x q1; y q2; x q3; z q4; }\ngate pauli_140709000661008(param0) q0,q1,q2,q3,q4 { z q0; x q1; y q2; z q3; z q4; }\ngate pauli_140709000814224(param0) q0,q1,q2,q3,q4 { y q2; z q4; }\ngate pauli_140709008254416(param0) q0,q1,q2,q3,q4 { y q2; z q3; z q4; }\ngate pauli_140709010876432(param0) q0,q1,q2,q3,q4 { y q1; z q2; x q3; }\ngate pauli_140709007204880(param0) q0,q1,q2,q3,q4 { y q1; z q2; x q3; }\nqreg q[5];\npauli(ZXIXY) q[0],q[1],q[2],q[3],q[4];\nh q[0];\nx q[1];\nz q[2];\nid q[3];\nz q[4];\ngate__ZXIXY q[0],q[1],q[2],q[3],q[4];\nbarrier q[0],q[1],q[2],q[3],q[4];\npauli_140709000663440(XZZXX) q[0]

In [ ]:
me = qk.QuantumCircuit(5)
me.pauli(pauli_string = "XX",qubits = [1,3])
print(me)

                   
q_0: ──────────────
     ┌────────────┐
q_1: ┤0           ├
     │            │
q_2: ┤  Pauli(XX) ├
     │            │
q_3: ┤1           ├
     └────────────┘
q_4: ──────────────
                   


In [ ]:
qasm_str = circuit_ensemble[0].qasm()
qk.QuantumCircuit.from_qasm_str(qasm_str)

QASM2ParseError: "<input>:4,40: identifiers cannot start with capital letters except for the builtins 'U' and 'CX'"